In [1]:
%load_ext autoreload
%autoreload 2

from typing import List, Dict, Any
from dotenv import load_dotenv
from langsmith import Client
from openai import OpenAI
from tqdm import tqdm
import json
import rag_helpers as rh

load_dotenv()

openai_client = rh.get_openai_client()
langsmith_client = Client()

In [2]:
from llama_index.core import SimpleDirectoryReader
from llama_index.readers.file import FlatReader

docs = SimpleDirectoryReader(
    input_files=[rh.TESLA_DOC],
    filename_as_id=True,
    file_extractor={
        ".md": FlatReader()
    }
).load_data()

from llama_index.core.node_parser import SentenceSplitter

parser = SentenceSplitter(
    chunk_size=2000,
    chunk_overlap=400
)

nodes = parser.get_nodes_from_documents(docs)

In [3]:
print(f"\nTotal number of chunks: {len(nodes)}\n\n")

print("\nFirst 3 chunks:")
for i in range(min(3, len(nodes))):
    print(f"\n--- Chunk {i+1} ---")
    print(nodes[i].text)
    print("\n" + "="*80)


Total number of chunks: 112



First 3 chunks:

--- Chunk 1 ---
Tesla logo

# OWNER'S MANUAL

Image of Tesla Cybertruck

Software version: 2024.33.15
North America
---
# YOUR OWNER'S MANUAL

For the latest and greatest information that is customized to your vehicle, view the Owner's Manual on your vehicle's touchscreen by touching the app launcher and then selecting the Manual app. The information is specific to your vehicle depending on the features you purchased, vehicle configuration, market region, and software version. In contrast, owner information that is provided by Tesla elsewhere is updated as necessary and may not contain information unique to your vehicle.

## RELEASE NOTES

Information about new features is displayed on the touchscreen after a software update, and can be viewed at any time by choosing the Release Notes tab in the Manual app, or by touching Controls > Software > Release Notes. If the content in the Owner's Manual on how to use your vehicle conflicts with i

In [4]:
# Leverage multiple prompts to generate a diverse set of questions

FACTUAL_PROMPT = """Generate 2-3 questions that real Cybertruck owners would actually type into a search bar or ask in an owners' forum. These should feel completely natural and conversational.

Write questions as if they were being typed into a search bar or asked in a forum. For example:

Instead of:
- "How do I activate the climate control system?"
- "What should I do if the touchscreen becomes unresponsive?"
- "How does one optimize range in cold weather?"

Write:
- "how to turn on AC in cybertruck"
- "screen frozen - what now?"
- "battery draining fast in cold weather"

Make questions feel real by:
1. Using natural search patterns
   - "how to..."
   - "why is my..."
   - "help with..."
2. Including context and emotion
   - "stuck at supercharger"
   - "help! frunk won't open"
   - "confused about ride height settings"
3. Writing like real people
   - Use contractions (I'm, won't, can't)
   - OK to use incomplete sentences
   - Include emotional context ("Help!", "Confused about...", "Worried about...")
4. Adding situational details
   - "in rain"
   - "with kids"
   - "while camping"

For each question, evaluate its real-world relevance:
- "common": Everyday, urgent needs:
  * "trunk won't close"
  * "phone key not working"
  * "what's this warning light mean"

- "rare": Occasional situations:
  * "winterizing cybertruck"
  * "car wash settings?"
  * "towing setup help"

- "unlikely": Technical/administrative:
  * Manual details
  * Specifications
  * Legal info

Text: {text}

Provide your response in the following JSON format:
{{
    "questions": [
        {{
            "question": "Natural, search-like question",
            "answer": "Clear, helpful answer",
            "supporting_text": "Relevant excerpt from source text",
            "question_type": "factual",
            "relevance_level": "common|rare|unlikely",
            "relevance_reasoning": "Brief explanation of why this question fits the chosen relevance level"
        }}
    ]
}}"""

REASONING_PROMPT = """Generate 2-3 questions that real Cybertruck owners would ask when trying to understand how features work together or make decisions about using their vehicle. These should feel like real forum posts or search queries.

Write questions as if they were being posted in an owners' forum. For example:

Instead of:
- "What is the optimal charging strategy?"
- "How does ambient temperature affect range?"
- "What are the considerations for child safety?"

Write:
- "best way to charge for long road trip?"
- "losing tons of range in cold - what helps?"
- "safest seats for car seats?"

Make questions feel real by:
1. Using natural patterns
   - "better to..."
   - "best way to..."
   - "tips for..."
2. Including context and emotion
   - "worried about range"
   - "confused about charging"
   - "need advice on settings"
3. Writing like real people
   - Use contractions (I'm, won't, can't)
   - OK to use incomplete sentences
   - Include emotional context ("Help!", "Confused about...", "Worried about...")
4. Adding situational details
   - "for camping"
   - "in winter"
   - "with full family"

For each question, evaluate its real-world relevance:
- "common": Everyday decisions:
  * "faster charging vs battery life?"
  * "seat heaters or cabin heat?"
  * "best settings for commute"

- "rare": Occasional planning:
  * "road trip planning help"
  * "winter driving tips"
  * "towing affects on range?"

- "unlikely": Technical/theoretical:
  * System details
  * Technical specs
  * Legal considerations

Text: {text}

Provide your response in the following JSON format:
{{
    "questions": [
        {{
            "question": "Natural, forum-style question",
            "answer": "Practical, helpful answer",
            "supporting_text": "Relevant excerpt from source text",
            "question_type": "reasoning",
            "relevance_level": "common|rare|unlikely",
            "relevance_reasoning": "Brief explanation of why this question fits the chosen relevance level"
        }}
    ]
}}"""

In [5]:
# Generate a small number of questions to test the prompts
# After inspecting the generated questions, we can adjust the prompts,
# if necessary.

response = openai_client.chat.completions.create(
    model=rh.OPENAI_MODEL,
    messages=[
        {"role": "system", "content": "You are a skilled question generator."},
        {"role": "user", "content": FACTUAL_PROMPT.format(text=nodes[0].text)}
    ],
    response_format={"type": "json_object"}
)

factual_questions = json.loads(response.choices[0].message.content)
print("Factual Questions Generated:")
print(json.dumps(factual_questions, indent=2))

# Test reasoning questions
response = openai_client.chat.completions.create(
    model=rh.OPENAI_MODEL,
    messages=[
        {"role": "system", "content": "You are a skilled question generator."},
        {"role": "user", "content": REASONING_PROMPT.format(text=nodes[0].text)}
    ],
    response_format={"type": "json_object"}
)

reasoning_questions = json.loads(response.choices[0].message.content)
print("\nReasoning Questions Generated:")
print(json.dumps(reasoning_questions, indent=2))


Factual Questions Generated:
{
  "questions": [
    {
      "question": "how do i open the cybertruck frunk?",
      "answer": "To open the powered frunk, use the app launcher on the vehicle's touchscreen and select the option for the frunk. Alternatively, you can use your key fob or the Tesla mobile app.",
      "supporting_text": "Powered Frunk...................................................................58 Opening and Closing.................................30",
      "question_type": "factual",
      "relevance_level": "common",
      "relevance_reasoning": "Many Cybertruck owners will need to know how to access the frunk for storage purposes, making this a common inquiry."
    },
    {
      "question": "why is my cybertruck battery draining fast in cold weather?",
      "answer": "Cold weather can increase energy consumption due to battery heating and climate control. To optimize range in cold conditions, precondition your Cybertruck while it is still plugged in.",
      "su

In [6]:
# Sample random chunks of the document to generate evaluation
# questions.

import random
NUM_CHUNKS = 10

random_chunks = random.sample(nodes, NUM_CHUNKS)
print(f"Selected {len(random_chunks)} random chunks")

# 2. Generate Questions for Random Chunks
candidate_examples = []

for node in tqdm(random_chunks):
    # Generate factual questions
    factual_response = openai_client.chat.completions.create(
        model=rh.OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are a skilled question generator."},
            {"role": "user", "content": FACTUAL_PROMPT.format(text=node.text)}
        ],
        response_format={"type": "json_object"}
    )
    
    # Generate reasoning questions
    reasoning_response = openai_client.chat.completions.create(
        model=rh.OPENAI_MODEL,
        messages=[
            {"role": "system", "content": "You are a skilled question generator."},
            {"role": "user", "content": REASONING_PROMPT.format(text=node.text)}
        ],
        response_format={"type": "json_object"}
    )
    
    # Parse responses
    factual_questions = json.loads(factual_response.choices[0].message.content)["questions"]
    reasoning_questions = json.loads(reasoning_response.choices[0].message.content)["questions"]
    
    # Format and store
    for question in factual_questions + reasoning_questions:
        example = {
            "question": question["question"],
            "answer": question["answer"],
            "metadata": {
                "chunk_id": node.node_id,
                "question_type": question["question_type"],
                "supporting_text": question["supporting_text"],
                "relevance_level": question["relevance_level"],
                "source_position": node.start_char_idx if hasattr(node, 'start_char_idx') else None,
                "filename": node.metadata.get("filename", "unknown")
            }
        }
        candidate_examples.append(example)

# 3. Filter for Common Questions
common_examples = [ex for ex in candidate_examples 
                  if ex["metadata"]["relevance_level"] == "common"]
print(f"\nFound {len(common_examples)} common questions")

Selected 10 random chunks


100%|██████████| 10/10 [01:46<00:00, 10.68s/it]


Found 42 common questions


In [9]:
print(len(candidate_examples))
print(json.dumps(candidate_examples[3], indent=2))

59
{
  "question": "lane assist acting up in fog - anyone else had this?",
  "answer": "Fog can interfere with Lane Assist performance by preventing cameras from seeing the lane markers clearly. Make sure your cameras are clean and try to drive attentively, as poor visibility conditions can reduce the feature's effectiveness.",
  "metadata": {
    "chunk_id": "9bc5a39d-40c4-43aa-94fe-aa7bb4d41ab9",
    "question_type": "reasoning",
    "supporting_text": "WARNING: Ensure all cameras are clean and free of obstructions... environmental conditions such as rain and faded lane markings can affect Autopilot performance.",
    "relevance_level": "common",
    "source_position": 376081,
    "filename": "Tesla Cybertruck Owners Manual.md"
  }
}


In [ ]:
import prompts
